### Configuração de Ambiente:

Baixar o arquivo csv, caso não tenha no projeto:

In [ ]:
import subprocess
import sys

# Tenta importar gdown, se não conseguir, instala
try:
    import gdown
except ImportError:
    print("📥 gdown não encontrado. Instalando automaticamente...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])
    import gdown  # Tenta importar novamente após a instalação

# Baixar o arquivo do Google Drive

file_id = "1VbB_fvowkgNoWcLADi1mEfFjWQI6nBjp"
url = f"https://drive.google.com/uc?id={file_id}"
output = "dados/SpSafe_2022.csv"

gdown.download(url, output, quiet=False)


Leitura do arquivo

In [ ]:
import pandas as pd
df = pd.read_csv('dados/SpSafe_2022.csv', delimiter=';')
pd.set_option('display.max_columns', None)


In [ ]:
#Conventendo a coluna DATA_OCORRENCIA para datetime para ser reconhecido como data pelo pandas
df['DATA_OCORRENCIA'] = pd.to_datetime(df['DATA_OCORRENCIA'])

In [ ]:
#Criação de coluna com o dia da semana
df.insert(df.columns.get_loc('DATA_OCORRENCIA') + 1, 'DIA_SEMANA', df['DATA_OCORRENCIA'].dt.day_name())

In [ ]:
#Convertendo os dias da semana para português
dias_semana = {
    'Monday': 'SEGUNDA-FEIRA', 'Tuesday': 'TERÇA-FEIRA', 'Wednesday': 'QUARTA-FEIRA',
    'Thursday': 'QUINTA-FEIRA', 'Friday': 'SEXTA-FEIRA', 'Saturday': 'SÁBADO', 'Sunday': 'DOMINGO'
}
df['DIA_SEMANA'] = df['DATA_OCORRENCIA'].dt.day_name().map(dias_semana)


Imprimindo resultado:

In [ ]:
df

### Incidentes:

Criação do campo:

In [ ]:
import pandas as pd

# Criar uma coluna combinada para o local de elaboração
df['LOCAL_ELABORACAO'] = df['DELEGACIA_ELABORACAO'].fillna(
                          df['DEPARTAMENTO_ELABORACAO'].fillna(
                          df['SECCIONAL_ELABORACAO']))

# Agora use esta coluna única no agrupamento
dados_incidente = ['DATA_OCORRENCIA', 'PERIODO_OCORRENCIA', 
                'LOGRADOURO', 'NUMERO_LOGRADOURO', 'BAIRRO', 'UF', 
                  'TIPO_LOCAL', 'LOCAL_ELABORACAO','DIA_SEMANA']

df['INCIDENTES_ID'] = df.groupby(dados_incidente).ngroup() + 1

# Verificar um incidente específico
df

Quantidade de Incidentes:

In [ ]:
# Filtrar apenas registros que pertencem a incidentes (excluindo possíveis registros sem agrupamento)
dados_com_incidentes = df[df['INCIDENTES_ID'].notna()]

# Mostrar esses dados
dados_com_incidentes.nunique()